In [1]:
# Запуск асинхронного чата с MCP
# await chat_with_llms_and_mcp()

In [2]:
import os
import asyncio
import json
from typing import Any, Dict, List

from rich.console import Console
from rich.panel import Panel
from rich.markdown import Markdown

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage

from mcp import ClientSession
from mcp.client.stdio import StdioServerParameters, stdio_client

import dotenv

dotenv.load_dotenv(dotenv.find_dotenv())

console = Console()

# Путь к локально собранному MCP серверу Airbnb
MCP_SERVER_PATH = os.path.join(
    "/Users",
    "dmitriikot",
    "Documents",
    "lectures",
    "HomeWork",
    "9_api",
    "mcp-server-airbnb",
    "dist",
    "index.js",
)


class MCPToolManager:
    """Управление подключением к MCP серверу и вызовами инструментов."""

    def __init__(self, command: str, args: List[str]):
        self.command = command
        self.args = args
        self._stdio_context = None
        self._session = None
        self._cached_tools = None

    async def connect(self) -> None:
        if self._session is not None:
            return

        params = StdioServerParameters(command=self.command, args=self.args)
        self._stdio_context = stdio_client(params)
        read_stream, write_stream = await self._stdio_context.__aenter__()
        
        self._session = ClientSession(read_stream, write_stream)
        await self._session.__aenter__()
        
        init_result = await self._session.initialize()
        try:
            server_info = getattr(init_result, 'serverInfo', None)
            if server_info:
                server_name = getattr(server_info, 'name', 'MCP Server')
                server_version = getattr(server_info, 'version', 'unknown')
                console.print(f"[dim]MCP сервер инициализирован: {server_name} v{server_version}[/dim]")
        except Exception:
            console.print("[dim]MCP сервер инициализирован[/dim]")
        self._cached_tools = None

    async def list_tools(self):
        await self.connect()
        if self._cached_tools is None:
            self._cached_tools = await self._session.list_tools()
        return self._cached_tools

    async def get_langchain_tools(self) -> List[Dict[str, Any]]:
        tools = await self.list_tools()
        langchain_tools: List[Dict[str, Any]] = []
        for tool in tools.tools:
            schema = tool.inputSchema or {"type": "object", "properties": {}}
            langchain_tools.append(
                {
                    "type": "function",
                    "function": {
                        "name": tool.name,
                        "description": tool.description or "",
                        "parameters": schema,
                    },
                }
            )
        return langchain_tools

    async def call_tool(self, name: str, arguments: Dict[str, Any]) -> Any:
        await self.connect()
        return await self._session.call_tool(name, arguments)

    async def close(self) -> None:
        if self._session:
            try:
                await self._session.__aexit__(None, None, None)
            except Exception:
                pass
            self._session = None
        if self._stdio_context:
            try:
                await self._stdio_context.__aexit__(None, None, None)
            except Exception:
                pass
            self._stdio_context = None
        self._cached_tools = None


def format_tool_result(result: Any) -> str:
    """Преобразует MCP ответ в строку для отображения и истории."""
    if getattr(result, "structuredContent", None):
        try:
            return json.dumps(result.structuredContent, ensure_ascii=False, indent=2)
        except (TypeError, ValueError):
            return str(result.structuredContent)

    text_chunks: List[str] = []
    if getattr(result, "content", None):
        for block in result.content:
            text = getattr(block, "text", None) or block.get("text") if isinstance(block, dict) else None
            if text:
                text_chunks.append(text)
    if text_chunks:
        return "\n".join(text_chunks)

    return str(result)


def get_llm_clients() -> Dict[str, ChatOpenAI]:
    llm1 = ChatOpenAI(
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=os.getenv("OPENAI_BASE_URL"),
        model="Qwen/Qwen3-Next-80B-A3B-Instruct",
    )

    llm2 = ChatOpenAI(
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=os.getenv("OPENAI_BASE_URL"),
        model="GigaChat/GigaChat-2-Max",
    )

    return {"llm1": llm1, "llm2": llm2}


def test_llms() -> None:
    llms = get_llm_clients()

    console.print(Panel("[bold cyan]Тестирование двух LLM[/bold cyan]", border_style="cyan"))

    test_messages = [
        SystemMessage(content="Ты очень хочешь всем помогать"),
        HumanMessage(content="Привет!"),
    ]

    console.print("\n[bold yellow]LLM 1:[/bold yellow]")
    response1 = llms["llm1"].invoke(test_messages)
    console.print(Markdown(response1.content))

    console.print("\n[bold yellow]LLM 2:[/bold yellow]")
    response2 = llms["llm2"].invoke(test_messages)
    console.print(Markdown(response2.content))


async def chat_with_llms_and_mcp() -> None:
    """Асинхронный чат с LLM и реальными MCP инструментами."""
    llms = get_llm_clients()
    active_llm = "llm2"

    mcp_manager = MCPToolManager(
        command="node",
        args=[MCP_SERVER_PATH, "--ignore-robots-txt"],
    )
    console.print("[dim]Пробую подключиться к MCP серверу...[/dim]")

    try:
        langchain_tools = await asyncio.wait_for(
            mcp_manager.get_langchain_tools(),
            timeout=30,
        )
        mcp_enabled = True
        console.print(
            Panel(
                f"[bold green]Подключено инструментов MCP: {len(langchain_tools)}[/bold green]",
                border_style="green",
            )
        )
    except asyncio.TimeoutError:
        await mcp_manager.close()
        console.print(
            Panel(
                "[bold yellow]MCP не подключен: таймаут подключения[/bold yellow]",
                border_style="yellow",
            )
        )
        langchain_tools = []
        mcp_enabled = False
    except Exception as e:
        await mcp_manager.close()
        console.print(
            Panel(
                f"[bold yellow]MCP не подключен: {str(e)}[/bold yellow]",
                border_style="yellow",
            )
        )
        langchain_tools = []
        mcp_enabled = False

    history: Dict[str, List[AIMessage | HumanMessage | SystemMessage | ToolMessage]] = {
        "llm1": [SystemMessage(content="Ты полезный ассистент с доступом к инструментам Airbnb")],
        "llm2": [SystemMessage(content="Ты полезный ассистент с доступом к инструментам Airbnb")],
    }

    runnable_cache: Dict[str, Any] = {}
    if mcp_enabled and langchain_tools:
        console.print("[dim]Привязываю инструменты к обеим моделям...[/dim]")
        runnable_cache["llm1"] = llms["llm1"].bind_tools(langchain_tools)
        runnable_cache["llm2"] = llms["llm2"].bind_tools(langchain_tools)
        console.print("[dim]Обе модели готовы к работе с инструментами[/dim]")

    try:
        while True:
            user_input = await asyncio.to_thread(input, f"\n({active_llm}) Вы: ")

            if not user_input.strip():
                continue

            if user_input.strip() == "/exit":
                console.print("[bold red]Гудбай[/bold red]")
                break

            if user_input.strip() == "/swap":
                active_llm = "llm2" if active_llm == "llm1" else "llm1"
                console.print(f"[bold cyan]Переключено на {active_llm}[/bold cyan]")
                continue

            if user_input.strip() == "/clear":
                for key in history:
                    system_msg = history[key][0]
                    history[key] = [system_msg]
                console.print("[bold yellow]История очищена[/bold yellow]")
                continue

            if user_input.strip() == "/model":
                console.print(f"[bold cyan]Активная модель: {active_llm}[/bold cyan]")
                console.print(f"[dim]MCP {'включен' if mcp_enabled else 'выключен'}[/dim]")
                continue

            if user_input.strip() == "/tools":
                if mcp_enabled:
                    tools_info = await mcp_manager.list_tools()
                    console.print("[bold cyan]Доступные MCP инструменты:[/bold cyan]")
                    for tool in tools_info.tools:
                        console.print(f"  • {tool.name}: {tool.description}")
                else:
                    console.print("[yellow]MCP инструменты недоступны[/yellow]")
                continue

            history[active_llm].append(HumanMessage(content=user_input))
            console.print(f"[dim]({active_llm}) думаю...[/dim]")

            if mcp_enabled and langchain_tools and active_llm in runnable_cache:
                response: AIMessage = await runnable_cache[active_llm].ainvoke(history[active_llm])
            else:
                response = await llms[active_llm].ainvoke(history[active_llm])

            if getattr(response, "tool_calls", None):
                console.print("[bold magenta]🔧 Модель вызывает инструменты...[/bold magenta]")
                history[active_llm].append(response)

                for tool_call in response.tool_calls:
                    tool_name = tool_call["name"]
                    tool_args = tool_call.get("args", {}) or {}
                    console.print(f"[yellow]Вызов: {tool_name}({tool_args})[/yellow]")

                    try:
                        tool_result = await mcp_manager.call_tool(tool_name, tool_args)
                        rendered = format_tool_result(tool_result)
                        console.print(f"[dim]Результат MCP:[/dim]\n{rendered}")
                    except Exception as tool_error:
                        rendered = f"Ошибка MCP: {tool_error}"
                        console.print(f"[bold red]{rendered}[/bold red]")

                    history[active_llm].append(
                        ToolMessage(content=rendered, tool_call_id=tool_call["id"])
                    )

                console.print(f"[dim]({active_llm}) формирую ответ...[/dim]")
                final_response = await llms[active_llm].ainvoke(history[active_llm])
                history[active_llm].append(final_response)
                console.print(Markdown(final_response.content))
            else:
                history[active_llm].append(response)
                console.print(Markdown(response.content))

    except KeyboardInterrupt:
        console.print("\n[bold yellow]Гудбай[/bold yellow]")
    finally:
        await mcp_manager.close()


def chat_with_llms() -> None:
    """Синхронный чат без MCP инструментов."""
    llms = get_llm_clients()
    active_llm = "llm1"

    history: Dict[str, List[AIMessage | HumanMessage | SystemMessage | ToolMessage]] = {
        "llm1": [SystemMessage(content="Ты полезный ассистент")],
        "llm2": [SystemMessage(content="Ты полезный ассистент")],
    }

    while True:
        try:
            user_input = input(f"\n({active_llm}) Вы: ")

            if not user_input.strip():
                continue

            if user_input.strip() == "/exit":
                console.print("[bold red]Гудбай[/bold red]")
                break

            if user_input.strip() == "/swap":
                active_llm = "llm2" if active_llm == "llm1" else "llm1"
                console.print(f"[bold cyan]Переключено на {active_llm}[/bold cyan]")
                continue

            if user_input.strip() == "/clear":
                for key in history:
                    system_msg = history[key][0]
                    history[key] = [system_msg]
                console.print("[bold yellow]История очищена[/bold yellow]")
                continue

            if user_input.strip() == "/model":
                console.print(f"[bold cyan]Активная модель: {active_llm}[/bold cyan]")
                continue

            history[active_llm].append(HumanMessage(content=user_input))
            console.print(f"[dim]({active_llm}) думаю...[/dim]")

            response = llms[active_llm].invoke(history[active_llm])
            history[active_llm].append(response)

            console.print(Markdown(response.content))

        except KeyboardInterrupt:
            console.print("\n[bold yellow]Гудбай[/bold yellow]")
            break
        except Exception as e:
            console.print(f"[bold red]Ошибка: {str(e)}[/bold red]")
            import traceback
            console.print(f"[dim]{traceback.format_exc()}[/dim]")

In [3]:
# await chat_with_llms()
await chat_with_llms_and_mcp()

Пробую подключиться к MCP серверу...

MCP сервер инициализирован: airbnb v0.1.3

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Подключено инструментов MCP: 2                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Привязываю инструменты к обеим моделям...

Обе модели готовы к работе с инструментами

...

(llm2) думаю...

Всего доброго! Обращайтесь, если потребуется помощь.

(llm2) думаю...

Прощайте! Если возникнут вопросы в будущем — я здесь, чтобы помочь вам.

CancelledError: 